In [1]:
import os
import sys
import itertools
import json

import pydocumentdb.documents as documents
import pydocumentdb.document_client as document_client
import pydocumentdb.errors as errors
import pydocumentdb.http_constants as http_constants

import pandas as pd; pd.options.display.max_rows = 10
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import numpy as np

import view.config as cfg
import view.storage as st
import view.normalization as norm

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from IPython.core.display import HTML
from IPython.display import display

In [2]:
HOST = cfg.settings.host
MASTER_KEY = cfg.settings.master_key
DATABASE_ID = cfg.settings.database_id
COLLECTION_ID = cfg.settings.collection_id

In [8]:
res_names = ["insufficients_offers"]

In [12]:
client = document_client.DocumentClient(HOST, {'masterKey': MASTER_KEY})

coll_link = st.GetCollectionLink(client, DATABASE_ID, COLLECTION_ID)
proc = [p for p in client.ReadStoredProcedures(coll_link, options=None) if p["id"]=="ReadResources"][0]
print proc["body"]
options = { 'enableScriptLogging': True }
params = {
    "resources": res_names, 
    "start": "2013-03-29T00:00:00+02:00",
    "end": "2013-03-29T23:30:00+01:00"
}
try:
    dodocs = client.ExecuteStoredProcedure(proc["_self"],json.dumps(params), options)
except Exception as ex:
    error = errors.JSONParseFailure(ex)
    print errors.JSONParseFailure(error)
print client.last_response_headers.get(http_constants.HttpHeaders.ScriptLogResults)

pd.DataFrame.from_dict([v  for d in dodocs[u'insufficients_offers'] for v in d["data"]])

function(params){
    var context = getContext();
    var collection = context.getCollection();
    var response = context.getResponse();
    
    var documents = {};

    var parameters = JSON.parse(params);

    for (var i = 0; i < parameters.resources.length; i++) {
        
        var resource = parameters.resources[i];
        
        var query = 'SELECT * FROM d WHERE d.resourcename = "' + resource + '"' +
        //' AND ( d["end_date"] >= "' + parameters.start + '")';
        ' AND ( (d["end_date"] >= "' + parameters.start + '") AND (d["start_date"] <= "'+ parameters.end +'"))';
        
        var queryDocs = collection.queryDocuments(collection.getSelfLink(), query, {},
        function(err, docs){
            
            // Error management
            if(err) throw new Error("Error" + err.message);
            if(docs.length == 0) documents[resource] = "Unable to parse documents";
                        
            documents[resource] = docs;

            for(var j=0;

""
